In [1]:
!git clone https://github.com/wandb/edu.git
%cd edu/rag-advanced
!pip install -qqq -r requirements.txt
%cd notebooks

import nltk

nltk.download("wordnet")
nltk.download("punkt")

Cloning into 'edu'...
remote: Enumerating objects: 4519, done.
remote: Counting objects: 100% (1402/1402), done.
remote: Compressing objects: 100% (540/540), done.
remote: Total 4519 (delta 1062), reused 1024 (delta 859), pack-reused 3117 (from 1)
Receiving objects: 100% (4519/4519), 28.03 MiB | 10.76 MiB/s, done.
Resolving deltas: 100% (2476/2476), done.
Updating files: 100% (461/461), done.
/content/edu/rag-advanced
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass("Please enter your COHERE_API_KEY")

Please enter your COHERE_API_KEY··········


In [3]:
import asyncio

import nest_asyncio

nest_asyncio.apply()
import numpy as np
import weave

from scripts.utils import display_source

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
WANDB_PROJECT = "rag-course"

weave_client = weave.init(WANDB_PROJECT)

Please login to Weights & Biases (https://wandb.ai/) to continue:


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Logged in as Weights & Biases user: shikhar_dave.
View Weave data at https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/weave


In [5]:
# download the `raw_data` Dataset from chapter 1
raw_data = weave.ref(
    "weave:///rag-course/rag-course/object/raw_data:nuZosGsP58MXKxhluN2hzvKK9XB8xSxlTuOBmEzWzLo"
).get()
raw_data = weave.ref("raw_data:v0").get()

# this is how we index into the data
print(raw_data.rows[:2])
raw_data = list(map(dict, raw_data.rows[:]))

[WeaveDict({'content': '---\ndescription: Log and visualize data without a W&B account\ndisplayed_sidebar: default\n---\n\n# Anonymous Mode\n\nAre you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first.\n\nAllow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)`\n\n:::info\n**Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com.\n:::\n\n### How does someone without an account see results?\n\nIf someone runs your script and you have to set `anonymous="allow"`:\n\n1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key f

In [6]:
from scripts.utils import (
    TOKENIZERS,
    get_special_tokens_set,
    length_function,
    tokenize_text,
)

# this is the function that will tokenize the text
display_source(tokenize_text)
# this is the function that calculates the number of tokens
display_source(length_function)

   1 def tokenize_text(text: str, model: str = "command-r") -> List[str]:                                          
   2     """                                                                                                       
   3     Tokenizes the given text using the specified model.                                                       
   4                                                                                                               
   5     Args:                                                                                                     
   6         text (str): The text to be tokenized.                                                                 
   7         model (str): The model to use for tokenization. Defaults to "command-r".                              
   8                                                                                                               
   9     Returns:                                                                                                  
  10         List[str]: A list of tokens.                                                                          
  11     """                                                                                                       
  12     co = cohere.Client(api_key=os.environ["COHERE_API_KEY"])                                                  
  13     return co.tokenize(text=text, model=model, offline=True)                                                  
  14                                                                                                               

   1 def length_function(text, model="command-r"):                                                                 
   2     """                                                                                                       
   3     Calculate the length of the tokenized text using the specified model.                                     
   4                                                                                                               
   5     Args:                                                                                                     
   6         text (str): The text to be tokenized and measured.                                                    
   7         model (str): The model to use for tokenization. Defaults to "command-r".                              
   8                                                                                                               
   9     Returns:                                                                                                  
  10         int: The number of tokens in the tokenized text.                                                      
  11     """                                                                                                       
  12     return len(tokenize_text(text, model=model).tokens)                                                       
  13                                                                                                               

In [7]:
for doc in raw_data[:]:
    doc["metadata"]["words"] = doc["metadata"].pop("raw_tokens")
    doc["metadata"]["tokens"] = length_function(doc["content"])
raw_data[:2]

[{'content': '---\ndescription: Log and visualize data without a W&B account\ndisplayed_sidebar: default\n---\n\n# Anonymous Mode\n\nAre you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first.\n\nAllow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)`\n\n:::info\n**Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com.\n:::\n\n### How does someone without an account see results?\n\nIf someone runs your script and you have to set `anonymous="allow"`:\n\n1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the ses

In [8]:
from scripts.preprocess import convert_contents_to_text, make_text_tokenization_safe

# this is the function that converts the markdown to html
display_source(convert_contents_to_text)
# this is the function that cleans the text
display_source(make_text_tokenization_safe)

   1 def convert_contents_to_text(contents: str) -> str:                                                           
   2     """                                                                                                       
   3     Converts the given markdown content to plain text.                                                        
   4                                                                                                               
   5     Args:                                                                                                     
   6         contents: A string containing the markdown content.                                                   
   7                                                                                                               
   8     Returns:                                                                                                  
   9         A string containing the plain text extracted from the markdown content.                               
  10     """                                                                                                       
  11     _, content = frontmatter.parse(contents)                                                                  
  12     markdown_document = markdown.markdown(                                                                    
  13         content,                                                                                              
  14         extensions=[                                                                                          
  15             "toc",                                                                                            
  16             "pymdownx.extra",                                                                                 
  17             "pymdownx.blocks.admonition",                                                                     
  18             "pymdownx.magiclink",                                                                             
  19             "pymdownx.blocks.tab",                                                                            
  20             "pymdownx.pathconverter",                                                                         
  21             "pymdownx.saneheaders",                                                                           
  22             "pymdownx.striphtml",                                                                             
  23             "pymdownx.highlight",                                                                             
  24             "pymdownx.pathconverter",                                                                         
  25             "pymdownx.escapeall",                                                                             
  26         ],                                                                                                    
  27     )                                                                                                         
  28     soup = BeautifulSoup(markdown_document, "html.parser")                                                    
  29                                                                                                               
  30     def remove_urls_a_tags_hrefs(soup):                                                                       
  31         """                                                                                                   
  32         Removes URLs, <a> tags, and <img> tags from the BeautifulSoup object.                                 
  33                                                                                                               
  34         Args:                                                                                                 
  35             soup: A BeautifulSoup object containing

   1 def make_text_tokenization_safe(                                                                              
   2     content: str, special_tokens_set: set = get_special_tokens_set()                                          
   3 ) -> str:                                                                                                     
   4     """                                                                                                       
   5     Makes the text safe for tokenization by removing special tokens.                                          
   6                                                                                                               
   7     Args:                                                                                                     
   8         content: A string containing the text to be processed.                                                
   9         special_tokens_set: A set of special tokens to be removed from the text.                              
  10                                                                                                               
  11     Returns:                                                                                                  
  12         A string with the special tokens removed.                                                             
  13     """                                                                                                       
  14                                                                                                               
  15     def remove_special_tokens(text: str) -> str:                                                              
  16         """                                                                                                   
  17         Removes special tokens from the given text.                                                           
  18                                                                                                               
  19         Args:                                                                                                 
  20             text: A string representing the text.                                                             
  21                                                                                                               
  22         Returns:                                                                                              
  23             The text with special tokens removed.                                                             
  24         """                                                                                                   
  25         for token in special_tokens_set:                                                                      
  26             text = text.replace(token, "")                                                                    
  27         return text                                                                                           
  28                                                                                                               
  29     cleaned_content = remove_special_tokens(content)                                                          
  30     return cleaned_content                                                                                    
  31                                                                                                               

In [9]:
special_tokens_set = get_special_tokens_set(TOKENIZERS["command-r"])
print(list(special_tokens_set)[:5])

parsed_data = []

for doc in raw_data:
    parsed_doc = doc.copy()
    content = convert_contents_to_text(doc["content"])
    parsed_doc["parsed_content"] = make_text_tokenization_safe(
        content, special_tokens_set=special_tokens_set
    )
    parsed_doc["metadata"]["parsed_tokens"] = length_function(
        parsed_doc["parsed_content"]
    )
    parsed_data.append(parsed_doc)
parsed_data[:2]

['<|EXTRA_2_TOKEN|>', '<|USER_2_TOKEN|>', '<|USER_9_TOKEN|>', '<|EXTRA_9_TOKEN|>', '<|END_OF_TURN_TOKEN|>']


[{'content': '---\ndescription: Log and visualize data without a W&B account\ndisplayed_sidebar: default\n---\n\n# Anonymous Mode\n\nAre you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first.\n\nAllow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)`\n\n:::info\n**Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com.\n:::\n\n### How does someone without an account see results?\n\nIf someone runs your script and you have to set `anonymous="allow"`:\n\n1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the ses

In [10]:
# build weave dataset
parsed_data = weave.Dataset(name="parsed_data", rows=parsed_data)

# publish the dataset
weave.publish(parsed_data)

📦 Published to https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/weave/objects/parsed_data/versions/GeyWaYowuhkL0QDd02lrbPdHxIxbIFfiY4EX3hKlIGM


ObjectRef(entity='shikhar_dave-iit-jodhpur', project='rag-course', name='parsed_data', digest='GeyWaYowuhkL0QDd02lrbPdHxIxbIFfiY4EX3hKlIGM', extra=())

In [11]:
# download the `parsed_data` Dataset
parsed_data = weave.ref(
    "weave:///rag-course/rag-course/object/parsed_data:UhWHAwXzvIcYaZ3X1x4eX2KDyYhCM4TPSsj8Oq8dLq4"
).get()

parsed_data = weave.ref("parsed_data:v0").get()

# this is how we index into the data
print(parsed_data.rows[:2])

parsed_data = list(map(dict, parsed_data.rows[:]))
parsed_data[:2]

[WeaveDict({'content': '---\ndescription: Log and visualize data without a W&B account\ndisplayed_sidebar: default\n---\n\n# Anonymous Mode\n\nAre you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first.\n\nAllow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)`\n\n:::info\n**Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com.\n:::\n\n### How does someone without an account see results?\n\nIf someone runs your script and you have to set `anonymous="allow"`:\n\n1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key f

[{'content': '---\ndescription: Log and visualize data without a W&B account\ndisplayed_sidebar: default\n---\n\n# Anonymous Mode\n\nAre you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first.\n\nAllow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)`\n\n:::info\n**Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com.\n:::\n\n### How does someone without an account see results?\n\nIf someone runs your script and you have to set `anonymous="allow"`:\n\n1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the ses

In [12]:
from scripts.chunking import chunk_documents

# this is the function that chunks the documents
display_source(chunk_documents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


   1 def chunk_documents(docs, chunk_size=CHUNK_SIZE):                                                             
   2     """                                                                                                       
   3     Chunks a list of documents into smaller pieces based on the specified chunk size.                         
   4                                                                                                               
   5     Args:                                                                                                     
   6         docs (list): A list of documents to be chunked. Each document should contain 'parsed_content' and 'met
   7         chunk_size (int): The desired size of each chunk. Defaults to CHUNK_SIZE.                             
   8                                                                                                               
   9     Returns:                                                                                                  
  10         list: A list of dictionaries, each containing 'cleaned_content' and 'metadata' for each chunk.        
  11     """                                                                                                       
  12     chuker = partial(chunk_document, chunk_size=chunk_size)                                                   
  13     chunked_data = map(chuker, docs)                                                                          
  14     chunked_docs = [                                                                                          
  15         item for sublist in tqdm(chunked_data, total=len(docs)) for item in sublist                           
  16     ]                                                                                                         
  17     return chunked_docs                                                                                       
  18                                                                                                               

In [13]:
sample_chunked_data = chunk_documents(parsed_data[:5])
sample_chunked_data[:2]

  0%|          | 0/5 [00:00<?, ?it/s]

[{'cleaned_content': 'Anonymous Mode\nAre you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first. Allow results to be logged in Anonymous Mode with wandb.init(anonymous="allow")\n:::info\nPublishing a paper? Please cite W&B, and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com.\n:::\nHow does someone without an account see results? If someone runs your script and you have to set anonymous="allow":\n\nAuto-create temporary account: W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the session. Log results quickly: The user can run and re-run the script, and automatically see results show up in the W&B dashboard UI.\nThese unclaimed anonymous runs will be available for 7 days.

In [14]:
# fetch the chunked data
chunked_data = weave.ref(
    "weave:///rag-course/rag-course/object/chunked_data:Ij9KThmiZQ9ljpCm8rVXTJlCaAbY2qC0zX6UJkBWHQ0"
).get()
chunked_data = weave.ref("chunked_data:latest").get()

# this is how we index into the data
print(chunked_data.rows[:2])

chunked_data = list(map(dict, chunked_data.rows[:]))

[WeaveDict({'content': '--- description: Log and visualize data without a W&B account displayed_sidebar: default --- # Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first. Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)` :::info **Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com. ::: ### How does someone without an account see results? If someone runs your script and you have to set `anonymous="allow"`: 1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the session. 2. **Log r

In [15]:
mean_chunk_size = np.mean([doc["metadata"]["parsed_tokens"] for doc in chunked_data])
std_chunk_size = np.std([doc["metadata"]["parsed_tokens"] for doc in chunked_data])
print(f"Mean chunk size: {mean_chunk_size}, Std chunk size: {std_chunk_size}")

KeyError: 'parsed_tokens'

In [16]:
from scripts.rag_pipeline import SimpleRAGPipeline
from scripts.response_generator import SimpleResponseGenerator
from scripts.retriever import BM25Retriever, TFIDFRetriever

display_source(BM25Retriever)

   1 class BM25Retriever(weave.Model):                                                                             
   2     """                                                                                                       
   3     A retriever model that uses BM25 for indexing and searching documents.                                    
   4                                                                                                               
   5     Attributes:                                                                                               
   6         index (bm25s.BM25): The BM25 index.                                                                   
   7         data (list): The data to be indexed.                                                                  
   8     """                                                                                                       
   9                                                                                                               
  10     index: bm25s.BM25 = bm25s.BM25()                                                                          
  11     data: list = None                                                                                         
  12                                                                                                               
  13     def index_data(self, data):                                                                               
  14         """                                                                                                   
  15         Indexes the provided data using BM25.                                                                 
  16                                                                                                               
  17         Args:                                                                                                 
  18             data (list): A list of documents to be indexed. Each document should be a dictionary              
  19                          containing a key 'cleaned_content' with the text to be indexed.                      
  20         """                                                                                                   
  21         self.data = data                                                                                      
  22         corpus = [doc["cleaned_content"] for doc in data]                                                     
  23                                                                                                               
  24         corpus_tokens = bm25s.tokenize(corpus, show_progress=False)                                           
  25                                                                                                               
  26         self.index.index(corpus_tokens, show_progress=False)                                                  
  27                                                                                                               
  28     @weave.op()                                                                                               
  29     def search(self, query, k=5):                                                                             
  30         """                                                                                                   
  31         Searches the indexed data for the given query using BM25.                                             
  32                                                                                                               
  33         Args:                                                                                                 
  34             query (str): The search query.                                                                    
  35             k (int): The number of top results to r

In [17]:
bm25_retriever = BM25Retriever()
bm25_retriever.index_data(chunked_data)

tfidf_retriever = TFIDFRetriever()
tfidf_retriever.index_data(chunked_data)

DEBUG:bm25s:Building index from IDs objects


In [18]:
INITIAL_PROMPT = open("prompts/initial_system.txt", "r").read()
response_generator = SimpleResponseGenerator(model="command-r", prompt=INITIAL_PROMPT)
bm25_rag_pipeline = SimpleRAGPipeline(
    retriever=bm25_retriever, response_generator=response_generator, top_k=5
)
tfidf_rag_pipeline = SimpleRAGPipeline(
    retriever=tfidf_retriever, response_generator=response_generator, top_k=5
)

In [19]:
from scripts.response_metrics import ALL_METRICS as RESPONSE_METRICS
from scripts.retrieval_metrics import ALL_METRICS as RETRIEVAL_METRICS

In [20]:
eval_dataset = weave.ref(
    "weave:///rag-course/dev/object/Dataset:Qj4IFICc2EbdXu5A5UuhkPiWgxM1GvJMIvXEyv1DYnM"
).get()

print("Number of evaluation samples: ", len(eval_dataset.rows))

Number of evaluation samples:  20


In [21]:
retrieval_evaluation = weave.Evaluation(
    name="Retrieval_Evaluation",
    dataset=eval_dataset,
    scorers=RETRIEVAL_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"], "k": 5},
)
bm25_retrieval_scores = asyncio.run(retrieval_evaluation.evaluate(bm25_retriever))
tfidf_retrieval_scores = asyncio.run(retrieval_evaluation.evaluate(tfidf_retriever))

Predict and score failed

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 290, in eval_example
    eval_row = await self.predict_and_score(model, example)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 464, in wrapper
    res, _ = await _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 276, in _call_async
    return handle_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 274, in _call_async
    res = await func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 227, in predict_and_score
    result = await async_call(score_fn, **score_args)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 464, in wrapper
    res, _ = await _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/o

Evaluated 1 of 20 examples

Predict and score failed

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 290, in eval_example
    eval_row = await self.predict_and_score(model, example)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 464, in wrapper
    res, _ = await _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 276, in _call_async
    return handle_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 274, in _call_async
    res = await func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 227, in predict_and_score
    result = await async_call(score_fn, **score_args)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 464, in wrapper
    res, _ = await _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/o

...

Evaluated 18 of 20 examples

Evaluated 19 of 20 examples

Evaluated 20 of 20 examples

Evaluation summary
{
    'compute_hit_rate': None,
    'compute_mrr': None,
    'compute_ndcg': None,
    'compute_map': None,
    'compute_precision': None,
    'compute_recall': None,
    'compute_f1_score': None,
    'llm_retrieval_scorer': None
}

🍩 https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/r/call/01927f64-3b45-7f61-aa2e-96f7ae18eb07


In [22]:
response_evaluations = weave.Evaluation(
    name="Response_Evaluation",
    dataset=eval_dataset,
    scorers=RESPONSE_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"]},
)
bm25_response_scores = asyncio.run(response_evaluations.evaluate(bm25_rag_pipeline))
tfidf_response_scores = asyncio.run(response_evaluations.evaluate(tfidf_rag_pipeline))

model_output failed

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 170, in predict_and_score
    model_output = await async_call(model_predict, **model_predict_args)
  File "/usr/lib/python3.10/asyncio/threads.py", line 25, in to_thread
    return await loop.run_in_executor(None, func_call)
  File "/usr/lib/python3.10/asyncio/futures.py", line 285, in __await__
    yield self  # This tells Task to wait for completion.
  File "/usr/lib/python3.10/asyncio/tasks.py", line 304, in __wakeup
    future.result()
  File "/usr/lib/python3.10/asyncio/futures.py", line 201, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 487, in wrapper
    res, _ = _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
  File "/usr/local/lib/python3

model_output failed

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 170, in predict_and_score
    model_output = await async_call(model_predict, **model_predict_args)
  File "/usr/lib/python3.10/asyncio/threads.py", line 25, in to_thread
    return await loop.run_in_executor(None, func_call)
  File "/usr/lib/python3.10/asyncio/futures.py", line 285, in __await__
    yield self  # This tells Task to wait for completion.
  File "/usr/lib/python3.10/asyncio/tasks.py", line 304, in __wakeup
    future.result()
  File "/usr/lib/python3.10/asyncio/futures.py", line 201, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 487, in wrapper
    res, _ = _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
  File "/usr/local/lib/python3

model_output failed

...

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 290, in eval_example
    eval_row = await self.predict_and_score(model, example)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 464, in wrapper
    res, _ = await _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 276, in _call_async
    return handle_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 274, in _call_async
    res = await func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 227, in predict_and_score
    result = await async_call(score_fn, **score_args)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 464, in wrapper
    res, _ = await _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/o

Evaluated 19 of 20 examples

Evaluated 20 of 20 examples

Evaluation summary
{
    'compute_diff': None,
    'compute_levenshtein': None,
    'compute_rouge': None,
    'compute_bleu': None,
    'llm_response_scorer': None
}

🍩 https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/r/call/01927f64-b3e1-7a31-8710-6e5b1b220dbe
